In [1]:
# Import libraries
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# For the global variables
FILE = 'folder/working_hours.txt'

TOTAL_MINUTES_HOUR = 60
DAILY_HOURS_TO_COVER = 8
NUM_WORKING_DAYS = 5

TOTAL_HOURS_TO_COVER = NUM_WORKING_DAYS * DAILY_HOURS_TO_COVER
MAX_MINUTES_RECOMMENDED = TOTAL_MINUTES_HOUR * DAILY_HOURS_TO_COVER
MAX_MINUTES_WEEKLY = TOTAL_MINUTES_HOUR * TOTAL_HOURS_TO_COVER
MAX_MINUTES_OVERTIME = TOTAL_MINUTES_HOUR * (DAILY_HOURS_TO_COVER + 1)

In [39]:
# Get the current time
def get_current_time():
    return str(datetime.now().hour) + ':' + str(datetime.now().minute)

# Return the total time covered in Xh and Ym
def get_hours_minutes(total_minutes):

    hours = int(total_minutes/60)
    minutes = int(total_minutes % 60)

    return str(hours) + 'h ' + str(minutes) + 'm'

# Find the difference of two timeframes
def get_duration(times):
    
    minutes_covered = 0

    # Each time gap has two outcomes: 
    # 1. 1400-1800 (240 minutes) 
    # 2. 0800- (Current time)
    for hours in times:
        hours = hours.rstrip('\n')
        
        # Split the string based on '-'
        (start_time, end_time) = hours.split('-')
        FMT = '%H:%M'

        end_time = end_time if end_time != '' else get_current_time()

        # Get the time difference
        diff = datetime.strptime(end_time, FMT) - datetime.strptime(start_time, FMT)
        minutes_covered += diff.seconds/60

    return int(minutes_covered)

def get_day_number():
    return datetime.today().weekday()

def find_average_time_remaining(total_covered, days_array):
    # Find the average time to cover on a daily basis to fulfill 40hours
    day_number = get_day_number()
    
    # 1. Get the total hours remaining
    # 2. Deduct today's hours to get hours covered previously (avoid dynamic average)
    total_time_exclude_today = MAX_MINUTES_WEEKLY - (total_covered - days_array[day_number]['minutes_covered'])
    days_remaining = NUM_WORKING_DAYS - day_number

    # We first find the average of minutes needed to be covered per day
    # Then we add the products of the previous command
    avg_mins, remainder = divmod(total_time_exclude_today, (days_remaining))
    total_avg = avg_mins + remainder
    
    # Find hours and minutes format, as our average
    avg_hour, avg_mins = divmod(total_avg, TOTAL_MINUTES_HOUR)
    time_to_cover = str(avg_hour) + 'h ' + str(avg_mins) + 'm'

    return total_avg, time_to_cover

def calculate_hours():
    total_covered = 0

    days_array = []
    minutes_array = []
    finishing_time_today = ''
    
    day_number = get_day_number()

    # Load text file
    with open(FILE) as file:

        # One line = Day, Time gap #1, Time gap #2, ..., Time gap n
        for index, line in enumerate(file):
            
            line_array = line.replace('\x00','').rstrip().split(',')
            
            day = line_array[0]
            day_coverage = line_array[1:]

            minutes_covered = get_duration(day_coverage)
            
            days_array.append({
                'day': day,
                'minutes_covered': minutes_covered,
                'hours_covered': get_hours_minutes(minutes_covered),
                'coverage': day_coverage
            })
            
            if index == day_number:
                
                remaining_today = MAX_MINUTES_RECOMMENDED - minutes_covered
                remaining_today = 0 if remaining_today <= 0 else remaining_today

            total_covered += minutes_covered

    # Calculate overall week stats
    total_covered = total_covered if total_covered <= MAX_MINUTES_WEEKLY else MAX_MINUTES_WEEKLY

    return (total_covered, days_array, remaining_today)

In [40]:
# Find:
# - the total hours covered
# - the days covered so far and the breakdown of minutes per day

(total_covered, days_array, remaining_today) = calculate_hours()
(avg_minutes, hours_minutes_string) = find_average_time_remaining(total_covered, days_array)

In [41]:
# Convert the days_array to dataframe
days_df = pd.DataFrame(days_array)
print(days_df)
print('\n\n')

   day  minutes_covered hours_covered                                 coverage
0  Mon              438        7h 18m  [09:12-11:12, 11:16-11:48, 12:06-16:52]
1  Tue              404        6h 44m               [08:37-11:38, 13:49-17:32]
2  Wed              527        8h 47m                            [08:10-16:57]
3  Thu              543         9h 3m               [08:10-11:56, 12:15-17:32]
4  Fri              105        1h 45m                                 [08:14-]





In [42]:
# Bar chart
fig = px.bar(days_df, 
             x='minutes_covered', y='day',
            hover_data=['coverage', 'hours_covered'],
            labels={
                'day': 'Day', 
                'minutes_covered': 'Total (Minutes)', 
                'hours_covered': 'Total (Hours)',
                'coverage': 'Day coverage'
            },
            orientation='h',
            height = 400)

# Recommended: 480 minutes
fig.add_shape(
        dict(
            type="line",
            x0=MAX_MINUTES_RECOMMENDED,
            x1=MAX_MINUTES_RECOMMENDED,
            y0=-0.5,
            y1=4.5,
            line=dict(
                color="Red",
                width=2,
                dash="dot"
            )
))

print(avg_minutes)

# Average time to cover
fig.add_shape(
        dict(
            type="line",
            x0=avg_minutes,
            x1=avg_minutes,
            y0=-0.5,
            y1=4.5,
            line=dict(
                color="#FECB52",
                width=2,
                dash="dot"
            ),
))


fig.update_layout(
    title_text='Weekly hours calculation (Individual Days)',
    yaxis=dict(autorange="reversed"),
    showlegend=True)

fig.update_shapes(dict(xref='x', yref='y'))

fig.show()

488


In [43]:
remaining_time = MAX_MINUTES_WEEKLY - total_covered if MAX_MINUTES_WEEKLY > total_covered else 0

# Convert to dataframe
total_calculation_df = pd.DataFrame([
    {
        'category': 'covered', 
        'amount': total_covered, 
        'amount_hrs': get_hours_minutes(total_covered)
    },
    {
        'category': 'remaining', 
        'amount': remaining_time,
        'amount_hrs': get_hours_minutes(remaining_time)
    }
])

In [44]:
# Stacked bar chart
covered_today = MAX_MINUTES_RECOMMENDED - remaining_today
finishing_time_today = datetime.now() + timedelta(minutes = remaining_today)
print('FINISHING TIME TODAY (8 HOURS): ', finishing_time_today.time().strftime("%H:%M:%S"))

fig = go.Figure()
fig.add_trace(go.Bar(
    y=['Today'],
    x=[covered_today],
    name='Covered',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=['Today'],
    x=[remaining_today],
    name='Remaining',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))

fig.update_layout(barmode='stack')
fig.show()

FINISHING TIME TODAY (8 HOURS):  16:15:03


In [45]:
remaining_time = MAX_MINUTES_WEEKLY - total_covered if MAX_MINUTES_WEEKLY > total_covered else 0

# Convert to dataframe
total_calculation_df = pd.DataFrame([
    {
        'category': 'covered', 
        'amount': total_covered, 
        'amount_hrs': get_hours_minutes(total_covered)
    },
    {
        'category': 'remaining', 
        'amount': remaining_time,
        'amount_hrs': get_hours_minutes(remaining_time)
    }
])

print(total_calculation_df)

    category  amount amount_hrs
0    covered    2017    33h 37m
1  remaining     383     6h 23m


In [46]:
# Pie chart
fig = go.Figure(go.Pie(
    name="",
    values = total_calculation_df['amount'],
    labels = total_calculation_df['category'],
    customdata = total_calculation_df['amount_hrs'],
    hovertemplate = "Category: %{label} <br>Total(minutes): %{value} </br>Total(hours): %{customdata}"

))

fig.update_layout(title_text='Overall weekly hours calculation: Remaining vs Covered')

fig.show()